In [2]:
import sys
sys.path.append('..')
from predict.predict import predict 
from cleansing.data_cleansing import preprocess_data, preprocess_data_classification

import pandas as pd
from dateutil import parser

X_train, _, _, _ = preprocess_data_classification()

list = ["시설물 종류_건축","시설물 종류_산업환경설비","시설물 종류_조경","시설물 종류_토목",
        "공공/민간 구분_공공","공공/민간 구분_민간",
        "날씨_강설","날씨_강우","날씨_강풍","날씨_맑음","날씨_안개","날씨_흐림",
        "공종_가설공사","공종_강구조물공사","공종_건축 토공사","공종_건축물 부대공사","공종_관공사","공종_관공사 부대공사","공종_교량공사","공종_금속공사","공종_기계설비공사","공종_기타","공종_댐 및 제방공사","공종_도로 및 포장공사","공종_도장공사","공종_말뚝공사","공종_목공사","공종_미장공사","공종_방수공사","공종_산업설비공사","공종_수장공사","공종_전기설비공사","공종_조경공사","공종_조적공사","공종_지반개량공사","공종_지반조사","공종_지붕 및 홈통공사","공종_지정공사","공종_창호 및 유리공사","공종_철골공사","공종_철근콘크리트공사","공종_철도 및 궤도공사","공종_타일 및 돌공사","공종_터널공사","공종_토공사","공종_통신설비공사","공종_특수 건축물공사","공종_프리캐스트 콘크리트공사","공종_하천공사","공종_항만공사","공종_해체 및 철거공사",
        "발생일시","공사비","공사기간","공정률","작업자수","설계안전성검토","기온","습도"]

df = pd.DataFrame({"class": list, "boolean": 0.0})
df =df.set_index("class")

#(1) 공사기간 (yy.mm.dd ~ yy.mm.dd)
start_day = '2020-06-01'
end_day = '2020-08-01'

#(2) 공사기간 계산
start_day = parser.parse(start_day)
end_day = parser.parse(end_day)
dur = (end_day - start_day).days
df.loc['공사기간'] = dur

#(3) 시설물종류 (건축~)
string = '시설물 종류_'
selected_facility = '건축'
df.loc[string + selected_facility] = 1.0
    
#(4) 공정률 (~%)
percent = 65
if percent:
    df.loc['공정률'] = float(percent)
else:
    df.loc['공정률'] = None 
    
#(6),(5) 공종 (철근콘크리트~)
selected_category = '가설공사'
string = '공종_'
df.loc[string + selected_category] = 1.0

#(7) 설계안전성검토 (대상,비대상)
selected_safety = "대상"
if (selected_safety == "대상"):
    df.loc['설계안전성검토'] = 1.0

#(8) 시설관리공사 (공공,민간)
selected_company = "공공"
if selected_company == "공공":
    df.loc['공공/민간 구분_공공'] = 1.0
    df.loc['공공/민간 구분_민간'] = 0.0
else:
    df.loc['공공/민간 구분_공공'] = 0.0
    df.loc['공공/민간 구분_민간'] = 1.0

#(9) 공사비 (~원)
cost = 10
df.loc['공사비'] = float(cost)

#(10) 작업자수 (~명)
person = 1000
df.loc['작업자수'] = float(person)

#(11, 12, 13) 날씨, 기온, 습도 -> 따로 입력X

# 기상청 데이터 연결 "기상청_단기예보 ((구)_동네예보) 조회서비스"
import requests
import xml.etree.ElementTree as ET
from datetime import datetime
string = '날씨_'
key = '맑음'

# API 요청 URL
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'

# 인증키
api_key = 'NminqLTNuSX5OFbyRamiOBFhuUBormib7/IeKYFKpWn1iXnxa1PEQ5IZAfJWebf8nOOb2FplMo5tdutaV6kUxQ=='

# 현재 날짜와 시간 가져오기
now = datetime.now()
date = now.strftime('%Y%m%d')  # 날짜 형식 변환
time = now.strftime('%H')+'00' # 시간 형식 변환  --> 여기서 오류, 10분 미만일 때 API 못불러옴

# # 사용자로부터 날짜와 시간 입력받기
# date = input('날짜를 입력하세요 (예: 20230608): ')
# time = input('시간을 입력하세요 (예: 1400): ')

# 요청 파라미터
params = {
    'serviceKey': api_key,
    'numOfRows': '10',  # 가져올 데이터 개수
    'dataType': 'XML',  # 응답 데이터 형식
    'base_date': date,  # 기준 날짜
    'base_time': time,  # 기준 시간
    'nx': '60',  # 위도
    'ny': '127'  # 경도
}

# API 요청 보내기
response = requests.get(url, params=params)
if (response.status_code != 200):
    params['base_time'] = '0000'
    response = requests.get(url, params=params)
xml_data = response.text

# XML 파싱
tree = ET.ElementTree(ET.fromstring(xml_data))
root = tree.getroot()

# 필요한 데이터 추출
items = root.findall('.//item')

found = False

for item in items:
    category = item.find('category').text
    if category == 'T1H':  # 기온(category=T1H) 데이터 추출
        temp = item.find('obsrValue').text
        found = True
    elif category == 'REH':  # 습도(category=REH) 데이터 추출
        humidity = item.find('obsrValue').text
        found = True
    elif category == "SKY": # 맑음=1, 흐림=4
        sky = item.find('obsrValue').text
        if sky == '4':
            key = '흐림'
        found = True
    elif category == "PTY": # 강설=3, 강우=1
        pty = item.find('obsrValue').text
        if pty == '3': 
            key = '강설'
        elif pty == '1':
            key = '강우'
        found = True
    elif category == "WSD": # 강풍>=9
        wsd = item.find('obsrValue').text 
        wsd = int(float(wsd))
        if wsd >= 9: 
            key = '강풍'
        found = True

df = df.squeeze()  # shape를 (59,)로 변환
print(predict(df))  # df 전체를 인자로 넣어 예측

OSError: dlopen(/Users/soyeon/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <D21A7969-4567-3BC7-94ED-6A9E83AE9D78> /Users/soyeon/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [1]:
print(df)

NameError: name 'df' is not defined